In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from subprocess import check_output
import warnings
warnings.filterwarnings('ignore')

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# Specify data directories
train_data_dir = 'train'
validation_data_dir = 'validation'

# Set up data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Specify batch size and image dimensions
batch_size = 32
img_height = 150
img_width = 150

# Generate batches of training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')  # Since it's a binary classification problem

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


2024-04-10 10:37:55.913019: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 10:37:55.941913: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 10:37:56.088058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 10:37:56.088095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 10:37:56.117203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Found 99 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten layer
model.add(Flatten())

# Dense layers
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [7]:
# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
3/3 [==============================] - 5s 2s/step - loss: 0.8302 - accuracy: 0.3433
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 0.6483 - accuracy: 0.6667
Epoch 3/10
3/3 [==============================] - 5s 1s/step - loss: 0.6258 - accuracy: 0.7015
Epoch 4/10
3/3 [==============================] - 5s 1s/step - loss: 0.6609 - accuracy: 0.6567
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 0.6565 - accuracy: 0.6562
Epoch 6/10
3/3 [==============================] - 6s 2s/step - loss: 0.6459 - accuracy: 0.6458
Epoch 7/10
3/3 [==============================] - 5s 1s/step - loss: 0.6482 - accuracy: 0.6418
Epoch 8/10
3/3 [==============================] - 5s 1s/step - loss: 0.6534 - accuracy: 0.6269
Epoch 9/10
3/3 [==============================] - 5s 2s/step - loss: 0.6311 - accuracy: 0.6866
Epoch 10/10
3/3 [==============================] - 5s 1s/step - loss: 0.6759 - accuracy: 0.6418


In [9]:
import numpy as np
from keras.preprocessing import image

# Load the image
img_path = '/home/snucse/validation/img53581406_b231a8ba4dd4214f_MG_R_ML_ANON.jpg'
img = image.load_img(img_path, target_size=(150, 150))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values

# Make prediction
prediction = model.predict(img_array)

# Convert prediction to class label
class_label = "malignant" if prediction < 0.5 else "benign"
print("Prediction:", prediction)
print("Class label:", class_label)


1/1 [==============================] - 0s 13ms/step
Prediction: [[0.63866806]]
Class label: benign


In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import local_binary_pattern

# Define function to extract LBP features from an image
def extract_lbp_features(image):
    # Resize image to a fixed size (e.g., 150x150 pixels)
    resized_image = resize(image, (150, 150))
    # Convert image to grayscale if needed
    if resized_image.ndim == 3:
        resized_image = resized_image.mean(axis=2)
    # Extract LBP features
    lbp_image = local_binary_pattern(resized_image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, 10), density=True)
    return hist

# Define function to load and extract features from all images in a directory
def load_and_extract_features_from_dir(directory):
    features_list = []
    labels_list = []
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.png')):
            image_path = os.path.join(directory, filename)
            image = imread(image_path)
            features = extract_lbp_features(image)
            features_list.append(features)
            labels_list.append(os.path.basename(directory))  # Use directory name as label
    return features_list, labels_list

# Specify directory containing images
dataset_dir = 'train'

# Load and extract features from all images in the directory
all_features = []
all_labels = []
for subdir in os.listdir(dataset_dir):
    if os.path.isdir(os.path.join(dataset_dir, subdir)):
        subdir_path = os.path.join(dataset_dir, subdir)
        features, labels = load_and_extract_features_from_dir(subdir_path)
        all_features.extend(features)
        all_labels.extend(labels)

# Convert lists to numpy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Print the shape of the feature matrix and label array
print("Feature matrix shape:", X.shape)


In [16]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define function to load and resize image
def load_and_resize_image(file_path):
    img = imread(file_path)
    img_resized = resize(img, (150, 150))  # Resize image to 150x150 pixels
    return img_resized.flatten()

# Define function to load images from directory
def load_images_from_dir(directory, label):
    images = []
    labels = []
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith(('.jpg', '.png'))]
    images = [load_and_resize_image(file_path) for file_path in file_paths]
    labels = [label] * len(images)
    return images, labels

# Specify directories containing images
malignant_dir = 'train/Malignant'
benign_dir = 'train/Benign'

# Load images and labels from directories
malignant_images, malignant_labels = load_images_from_dir(malignant_dir, 1)
benign_images, benign_labels = load_images_from_dir(benign_dir, 0)

# Concatenate malignant and benign images and labels
X = np.concatenate([malignant_images, benign_images])
y = np.concatenate([malignant_labels, benign_labels])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.35
Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.50      0.32         6
           1       0.57      0.29      0.38        14

    accuracy                           0.35        20
   macro avg       0.40      0.39      0.35        20
weighted avg       0.47      0.35      0.36        20

Confusion Matrix:
[[ 3  3]
 [10  4]]
